## GEO2R in Colab

Slightly modified version of the code produced by [GEO2R](https://www.ncbi.nlm.nih.gov/geo/geo2r/) for series [GSE161472](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE161472). Modifications done by Brona Brejova (marked as BB). Text commentary was also added by BB.

* To set up R as the language  `Connect`, `Change runtime type`, `Runtime type` choose `R`.
* To run all cells of the notebook: Main menu `Run all`.

More information about DESeq2 package see <https://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html>


## Installing libraries

First we need to install DESeq2 library to be used for analysis and to load it in R.

In [ ]:
# BB: install libraries as ubuntu packages
system("apt update")
system("apt install -y r-bioc-all r-bioc-geoquery r-bioc-deseq2")

In [ ]:
# Version info: R 4.2.2, Biobase 2.58.0, GEOquery 2.66.0, limma 3.54.0
################################################################
#   Differential expression analysis with DESeq2
library(DESeq2)

## Loading data from GEO database and performing basic transformations

We selected data from article Luo, X., Yin, J., Dwyer, D., Yamawaki, T., Zhou, H., Ge, H., Han, C.Y., Shkumatov, A., Snyder, K., Ason, B. and Li, C.M., 2021. [Chamber-enriched gene expression profiles in failing human hearts with reduced ejection fraction. ](https://doi.org/10.1038/s41598-021-91214-2) Scientific reports, 11(1), p.11839.

From all 84 experiments in this data set we selected samples coming from donors with non-failing hearts and separated them into samples from ventricles (19 samples) and atriums (17 samples). Ventricle is a larger chamber (komora) and atrium is a smaller chamber (predsieň). Samples from the left and right sides of the heart were combined.

In [ ]:
# load counts table from GEO
urld <- "https://www.ncbi.nlm.nih.gov/geo/download/?format=file&type=rnaseq_counts"
path <- paste(urld, "acc=GSE161472", "file=GSE161472_raw_counts_GRCh38.p13_NCBI.tsv.gz", sep="&");
## BB: renames tbl to geo_tbl
geo_tbl <- as.matrix(data.table::fread(path, header=T, colClasses="integer"), rownames="GeneID")

# load gene annotations 
apath <- paste(urld, "type=rnaseq_counts", "file=Human.GRCh38.p13.annot.tsv.gz", sep="&")
annot <- data.table::fread(apath, header=T, quote="", stringsAsFactors=F, data.table=F)
rownames(annot) <- annot$GeneID



## Setting parameters and running DESeq2 analysis

Table with results can be found in file table.tsv (see the folder icon in the left column menu). You can download it by right clicking on the file and selecting "Download".

In [ ]:
# sample selection
gsms <- paste0("X000000000XXXXXXXXXXX000000000XXXXXXXXXXXX11111111",
        "XXXXXXXXXXXX111111111XXXXXXXXXXXX")
sml <- strsplit(gsms, split="")[[1]]

# filter out excluded samples (marked as "X")
sel <- which(sml != "X")
sml <- sml[sel]
# BB: renamed tbl below to geo_tbl
tbl <- geo_tbl[ ,sel]


In [ ]:
# group membership for samples
gs <- factor(sml)
groups <- make.names(c("ventricle", "atrium"))
levels(gs) <- groups
sample_info <- data.frame(Group = gs, row.names = colnames(tbl))

# pre-filter low count genes
# keep genes with at least N counts > 10, where N = size of smallest group
keep <- rowSums( tbl >= 10 ) >= min(table(gs))
tbl <- tbl[keep, ]

ds <- DESeqDataSetFromMatrix(countData=tbl, colData=sample_info, design= ~Group)

ds <- DESeq(ds, test="Wald", sfType="poscount")

# extract results for top genes table
r <- results (ds, contrast=c("Group", groups[1], groups[2]), alpha=0.01, pAdjustMethod ="fdr")

tT <- r[order(r$padj)[1:250],] 
tT <- merge(as.data.frame(tT), annot, by=0, sort=F)

# BB: write the table to file instead of screen 
#tT <- subset(tT, select=c("GeneID","padj","pvalue","lfcSE","stat","log2FoldChange","baseMean","Symbol","Description"))
#write.table(tT, file=stdout(), row.names=F, sep="\t")
write.table(tT, file="table.tsv", row.names=F, sep="\t")


## Visualization of results

In [ ]:
plotDispEsts(ds, main="GSE161472 Dispersion Estimates")

# create histogram plot of p-values
hist(r$padj, breaks=seq(0, 1, length = 21), col = "grey", border = "white", 
         xlab = "", ylab = "", main = "GSE161472 Frequencies of padj-values")

# volcano plot
old.pal <- palette(c("#00BFFF", "#FF3030")) # low-hi colors
par(mar=c(4,4,2,1), cex.main=1.5)
plot(r$log2FoldChange, -log10(r$padj), main=paste(groups[1], "vs", groups[2]),
     xlab="log2FC", ylab="-log10(Padj)", pch=20, cex=0.5)
with(subset(r, padj<0.05 & abs(log2FoldChange) >= 0),
     points(log2FoldChange, -log10(padj), pch=20, col=(sign(log2FoldChange) + 3)/2, cex=1))
legend("bottomleft", title=paste("Padj<", 0.05, sep=""), legend=c("down", "up"), pch=20,col=1:2)

# MD plot
par(mar=c(4,4,2,1), cex.main=1.5)
plot(log10(r$baseMean), r$log2FoldChange, main=paste(groups[1], "vs", groups[2]),
     xlab="log10(mean of normalized counts)", ylab="log2FoldChange", pch=20, cex=0.5)
with(subset(r, padj<0.05 & abs(log2FoldChange) >= 0),
     points(log10(baseMean), log2FoldChange, pch=20, col=(sign(log2FoldChange) + 3)/2, cex=1))
legend("bottomleft", title=paste("Padj<", 0.05, sep=""), legend=c("down", "up"), pch=20,col=1:2)
abline(h=0)
palette(old.pal) # restore palette


## General expression data visualization

In [ ]:
################################################################
#   General expression data visualization
dat <- log10(counts(ds, normalized = T) + 1) # extract normalized counts

# box-and-whisker plot
lbl <- "log10(raw counts + 1)"
ord <- order(gs)  # order samples by group
palette(c("#1B9E77", "#7570B3", "#E7298A", "#E6AB02", "#D95F02",
          "#66A61E", "#A6761D", "#B32424", "#B324B3", "#666666"))
par(mar=c(7,4,2,1))
boxplot(dat[,ord], boxwex=0.6, notch=T, main="GSE161472", ylab="lg(norm.counts)", outline=F, las=2, col=gs[ord])
legend("topleft", groups, fill=palette(), bty="n")

# BB: umap omitted